## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [2]:
import main
import psycopg2 as pg
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432,
                        dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

#### 1)

In [86]:
# Lista filmów tej samej długości
df = pd.read_sql("""
SELECT film.length, COUNT(film.length)
FROM film
GROUP BY film.length
""", con=connection)
df

,length,count
0,116,4
1,184,8
2,87,6
3,71,7
4,68,5
...,...,...
135,148,7
136,129,7
137,130,6
138,143,7


#### 2)

In [5]:
# Przykładowi klienci mieszkający w mieście Londyn
main.client_from_city('London')

,city,first_name,last_name
0,London,Mattie,Hoffman
1,London,Cecil,Vines


#### 3)

In [6]:
df = pd.read_sql("SELECT AVG(rental_rate) FROM film", con=connection)
df

,avg
0,2.98


#### 4)

In [7]:
df = pd.read_sql("""
SELECT category.name, count(film.title) FROM CATEGORY
INNER JOIN film_category ON category.category_id = film_category.category_id
INNER JOIN film ON film_category.film_id = film.film_id
GROUP BY category.name
""", con=connection)
df

,name,count
0,Sports,74
1,Classics,57
2,New,63
3,Family,69
4,Comedy,58
5,Animation,66
6,Travel,57
7,Music,51
8,Drama,62
9,Horror,56


#### 5)

In [8]:
#Wyświetl liczbę wszystkich klientów pogrupowanych według kraju 
df = pd.read_sql("""
SELECT country.country, count(customer.customer_id)
FROM customer 
INNER JOIN address ON customer.address_id = address.address_id
INNER JOIN city ON address.city_id = city.city_id
INNER JOIN country ON city.country_id = country.country_id
GROUP BY country.country
""", con=connection)
df

,country,count
0,Bangladesh,3
1,Indonesia,14
2,Venezuela,7
3,Cameroon,2
4,Czech Republic,1
...,...,...
103,Tanzania,3
104,Poland,8
105,Greenland,1
106,French Guiana,1


#### 6)

In [9]:
# Sklep z klientami 100 < x < 300
df = pd.read_sql("""
SELECT store_id, count(customer_id) FROM customer
GROUP BY store_id
HAVING count(customer_id) BETWEEN 100 AND 300
""", con=connection)
df

,store_id,count
0,2,273


#### 7)

In [10]:
# Wszyscy klienci którzy oglądali filmy ponad 200h
df = pd.read_sql("""
SELECT customer.first_name, customer.last_name, SUM(film.length)
FROM
    customer, rental, inventory, film
WHERE
    customer.customer_id = rental.customer_id
    AND rental.inventory_id = inventory.inventory_id
    AND inventory.film_id = film.film_id
GROUP BY customer.customer_id
HAVING SUM(film.length) > 200
""", con=connection)
df

,first_name,last_name,sum
0,Wanda,Patterson,3446
1,Vivian,Ruiz,2649
2,Dan,Paine,2750
3,Priscilla,Lowe,3606
4,Guy,Brownlee,3309
...,...,...,...
594,Oscar,Aquino,2119
595,Mitchell,Westmoreland,3736
596,Judith,Cox,3515
597,Doris,Reed,2756


#### 8)

In [92]:
# Średnia wartość wypożyczenia filmu
df = pd.read_sql("SELECT AVG(amount) FROM payment", con=connection)
df

,avg
0,4.200606


#### 9)

In [93]:
# Średnia wartość długości filmów we wszystkich kategoriach
df = pd.read_sql("""
SELECT category.name, AVG(film.length) FROM category
INNER JOIN film_category ON category.category_id = film_category.category_id
INNER JOIN film ON film_category.film_id = film.film_id
GROUP BY category.name
""", con=connection)
df

,name,avg
0,Sports,128.202703
1,Classics,111.666667
2,New,111.126984
3,Family,114.782609
4,Comedy,115.827586
5,Animation,111.015152
6,Travel,113.315789
7,Music,113.647059
8,Drama,120.838710
9,Horror,112.482143


#### 10)

In [61]:
# Najdłuższe tytuły filmowe we wszystkich kategoriach 
df = pd.read_sql("""
SELECT
    new_name.category, film.title, LENGTH(film.title)
FROM (
    SELECT category.name AS category , MAX(LENGTH(film.title)) FROM category
    INNER JOIN film_category ON category.category_id = film_category.category_id
    INNER JOIN film ON film_category.film_id = film.film_id
    GROUP BY category.name
    ) new_name, category, film_category, film
WHERE
    category.category_id = film_category.category_id
    AND film.film_id = film_category.film_id
    AND new_name.category = category.name
    AND new_name.MAX = LENGTH(film.title)
ORDER BY category.name
""", con=connection)
df

,category,title,length
0,Action,Entrapment Satisfaction,23
1,Animation,Telemark Heartbreakers,22
2,Children,Heartbreakers Bright,20
3,Children,Microcosmos Paradise,20
4,Children,Sweethearts Suspects,20
5,Classics,Extraordinary Conquerer,23
6,Comedy,Trainspotting Strangers,23
7,Documentary,Intolerable Intentions,22
8,Documentary,Deliverance Mulholland,22
9,Drama,Goldfinger Sensibility,22


#### 11)

In [60]:
# Wynik się zgadza, jest to najdłuższy film
df = pd.read_sql("""
SELECT title, LENGTH(title) FROM film
ORDER BY LENGTH(title) DESC
LIMIT 1 
""", con=connection)
df

,title,length
0,Arachnophobia Rollercoaster,27


### Testowanie funkcji z pliku main

In [67]:
main.film_in_category(10)

,title,languge,category
0,Autumn Crow,English,Games
1,Bulworth Commandments,English,Games
2,Candles Grapes,English,Games
3,Chicago North,English,Games
4,Creatures Shakespeare,English,Games
...,...,...,...
56,Vanilla Day,English,Games
57,Videotape Arsenic,English,Games
58,Volcano Texas,English,Games
59,Wanda Chamber,English,Games


In [69]:
main.number_films_in_category(12)

,category,count
0,Music,51


In [74]:
main.number_film_by_length(52, 58)

,length,count
0,52,7
1,53,9
2,54,6
3,55,2
4,56,5
5,57,7
6,58,7


In [85]:
main.client_from_city('Czestochowa')

,city,first_name,last_name
0,Czestochowa,Sidney,Burleson


In [86]:
main.avg_amount_by_length(50)

,length,avg
0,50,3.810598


In [87]:
main.client_by_sum_length(1265)

,first_name,last_name,sum
0,Antonio,Meek,1451
1,Leona,Obrien,1588
2,Katherine,Rivera,1615
3,Tiffany,Jordan,1667
4,Jerome,Kenyon,1679
...,...,...,...
593,Wesley,Bull,4808
594,Clara,Shaw,4808
595,Tammy,Sanders,5065
596,Eleanor,Hunt,5360


In [91]:
main.category_statistic_length('Games')

,category,avg,sum,min,max
0,Games,127.836066,7798,57,185
